Index-Building:
The factors used include the following:
- planet mass (Mjup)
- planet radius (Rjup)
- planet temperature (K)
- planet density CALCULATE
- radiation intensity (W/m^2) CALCULATE
- star distance (pc)
- AND/OR HZ measurement CALCULATE
- star metallicity ?? fraction
- star mass (Msun)
- sun radius (Rsun)
- star age ?? Gy
- star effective temperature (K)

The index will consist of the following:
- planet mass +
planet radius +
planet temperature +
star distance (HZ) + 
radiation intensity

Weighing can be determined in 1 of 2 ways:
- use a correlation regression as done in Cobb Douglas
- use difference from "ideal" Earth on a normal distribution to determine the importance of outliers

1. Import data and packages

In [1]:
# import packages
import numpy as np
import pandas as pd
import math 
from math import pi
import matplotlib.pyplot as plt


# import data
data = pd.read_csv('C:/Users/21sko/Desktop/dissertation/data/exo_data.csv')

#display data
# print(data)

2. Calculate density

2.1 Convert: 
- M(jup) to M(earth)
- R(jup) to R(earth)

In [2]:
# M(jup) to M(earth)
def mjup_to_mearth(planet_mass_mjup):
    mass_mearth = planet_mass_mjup * 317.8 

    return mass_mearth

data['planet_mass_mearth'] = data.apply(lambda row: mjup_to_mearth(row['planet_mass_mjup']), axis = 1)



# R(jup) to R(earth)
def rjup_to_rearth(planet_radius_rjup):
    radius_rearth = planet_radius_rjup * 11.2

    return radius_rearth

data['planet_radius_rearth'] = data.apply(lambda row: rjup_to_rearth(row['planet_radius_rjup']), axis = 1)

# print(data)

2.2 Calculate volume 

In [3]:
# 4/3 * pi* r^3
def volume_formula(planet_radius_rearth):
    volume = (4/3) * np.pi * (planet_radius_rearth ** 3)

    return volume

data['planet_volume_rearth3'] = data.apply(lambda row: volume_formula(row['planet_radius_rearth']), axis = 1)

# print(data)

2.3 Calculate relative density

In [4]:
# D = M / V

def density_formula(planet_mass_mearth, planet_volume_rearth):
    density = (planet_mass_mearth) / (planet_volume_rearth)
    
    return density


data['planet_density_relative'] = data.apply(lambda row: density_formula(row['planet_mass_mearth'], row['planet_radius_rearth']), axis = 1)

# print(data)

3. Calculate Habitability Zone boundaries

3.1 Version 1: simpler

In [5]:
# Function to convert mass to luminosity 
def mass_to_luminosity(star_mass_msun):
    return star_mass_msun  ** 3.6

# Function to calculate habitable zone boundaries 
def calculate_habitable_zone(Luminosity_Lsun):
    inner_boundary = np.sqrt(Luminosity_Lsun / 1.1)
    outer_boundary = np.sqrt(Luminosity_Lsun / 0.53)
    return inner_boundary, outer_boundary 

# Convert mass to luminosity 
data['Luminosity_Lsun'] = data['star_mass_msun'].apply(mass_to_luminosity) 

# Calculate HZ boundaries and add them to the dataframe 
data[['HZ_inner_AU_s', 'HZ_outer_AU_s']] = data['Luminosity_Lsun'].apply( lambda luminosity: calculate_habitable_zone(luminosity) ).apply(pd.Series)

# Display the updated dataset to the user in a Jupyter-friendly format 
# print(data)

3.2 Version 2: More complex

In [6]:
# inner flux 
def s_inner(Teff):
    s_inner = 1.296 - 2.139e-4 * Teff + 4.19e-8 * (Teff)**2

    return s_inner

data['Flux_inner_HZ'] = data['star_teff_K'].apply(s_inner) 

# outer flux
def s_outer(Teff):
    s_outer = 0.234 - 1.319e-5 * Teff + 6.19e-10 * (Teff)**2

    return s_outer

data['Flux_outer_HZ'] = data['star_teff_K'].apply(s_outer) 

# inner boundary
def r_inner(Luminosity_Lsun, Flux_inner_HZ):
    r_inner = np.sqrt(Luminosity_Lsun / Flux_inner_HZ)

    return r_inner

data['HZ_inner_AU_2'] = data.apply(lambda row: r_inner(row['Luminosity_Lsun'], row['Flux_inner_HZ']), axis = 1)


# outer boundary
def r_outer(Luminosity_Lsun, Flux_outer_HZ):
    r_outer = np.sqrt(Luminosity_Lsun / Flux_outer_HZ)

    return r_outer

data['HZ_outer_AU_2'] = data.apply(lambda row: r_outer(row['Luminosity_Lsun'], row['Flux_outer_HZ']), axis = 1)

# print(data)

4. Calculate middle of HZ

In [10]:
def HZ_middle(HZ_inner, HZ_outer):
    HZ_middle = (HZ_inner + HZ_outer) / 2

    return HZ_middle

data['HZ_middle_AU_s'] = data.apply(lambda row: HZ_middle(row['HZ_inner_AU_s'], row['HZ_outer_AU_s']), axis = 1)
data['HZ_middle_AU_2'] = data.apply(lambda row: HZ_middle(row['HZ_inner_AU_2'], row['HZ_outer_AU_2']), axis = 1)


5. Radiation Intensity
- Calculate distance from planet to star in m

In [ ]:
def pc_to_m(star_distance_pc):
    star_distance_m = star_distance_pc * 3.086e+16

    return star_distance_m

data['star_distance_m'] = data['star_distance_pc'].apply(pc_to_m) 

# print(data)


In [8]:
# radiation = L / 4pi(r)^2 W/m^2
def radiation_I(luminosity, distance_m):
    radiation_I = luminosity / (4*np.pi*(distance_m**2))

    return radiation_I


data['radiation_I41'] = data.apply(lambda row: radiation_I(row['Luminosity_Lsun'], row['star_distance_m']), axis = 1)

In [11]:
print(data)



         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         

5. Calculate ideal values ??

- planet mass M(earth) == Earth
- planet radius R(earth) == Earth
- planet temperature (K) == Earth
- planet density == Earth
- radiation intensity (W/m^2) == Earth ?
- HZ measurement == normal distribution, y = middle of HZ; non ideal = | 2sigma | where 1.5 sigma == boundary 
- star age ?? Gy

5.1 HZ normal distribution

In [28]:
def HZ_normal_curve(data):

    lower_bound_values = []
    upper_bound_values = []

    for index, row in data.iterrows():
        midpoint = row["HZ_middle_AU_s"]
        outer_boundary = row["HZ_outer_AU_s"]

        # Calculate bounds for values exactly 2 std deviations away from the mean
        lower_bound = midpoint - 2 * ((outer_boundary - midpoint) / 1.5)
        upper_bound = midpoint + 2 * ((outer_boundary - midpoint) / 1.5)
        
        # Append values to lists
        lower_bound_values.append(lower_bound)
        upper_bound_values.append(upper_bound)
    
    # Add new columns to DataFrame
    data['LowerBoundValue'] = lower_bound_values
    data['UpperBoundValue'] = upper_bound_values
    
    return data

# Apply the function to your dataset
updated_data = HZ_normal_curve(data)

# Display the updated dataset with specific values at lower and upper bounds recorded
print(updated_data)




         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         